!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
pip install --no-cache-dir --index-url https://pypi.nvidia.com pytorch-quantization !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 ./ch transform --config configs/examples/toy_uniform_tensorRT.toml --load /root/mase/TensorRTDev/jsc-tiny_classification_jsc_2024-03-03/software/training_ckpts/best.ckpt --load-type pl
./ch train --config configs/examples/toy_uniform_tensorRT.toml
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

sudo apt update && sudo apt upgrade -y 

wget https://repo.continuum.io/archive/Anaconda3-2023.09-0-Linux-x86_64.sh 

bash Anaconda3-2023.09-0-Linux-x86_64.sh 

Close and reopen terminal

source /root/.bashrc

conda config --set auto_activate_base false

git clone https://github.com/mau-mar/mase/

cd mase

bash scripts/init-conda.sh

source /opt/conda/etc/profile.d/conda.sh

conda activate mase

conda config --add channels conda-forge

git checkout origin/mauro-tensorRT-integration

cuda-python
absl-py
scipy
prettytable
sphinx-glpi-theme

./ch train --config configs/examples/toy_uniform_tensorRT.toml

In [1]:
import sys
import logging
import os
from pathlib import Path
from pprint import pprint as pp

# # figure out the correct path
machop_path = Path(".").resolve().parent /"machop"
assert machop_path.exists(), "Failed to find machop at: {}".format(machop_path)
sys.path.append(str(machop_path))
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch_tensorrt

from torch.utils.tensorboard import SummaryWriter

import pytorch_quantization
from pytorch_quantization import nn as quant_nn
from pytorch_quantization import quant_modules
from pytorch_quantization.tensor_quant import QuantDescriptor
from pytorch_quantization import calib
from tqdm import tqdm

print(pytorch_quantization.__version__)

from chop.dataset import MaseDataModule, get_dataset_info
from chop.tools.logger import set_logging_verbosity

from chop.passes.graph import (
    save_node_meta_param_interface_pass,
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
    tensorrt_calibrate_transform_pass,
    tensorrt_train_transform_pass,
    tensorrt_quantize_transform_pass,
    tensorrt_analysis_pass,
    )
from chop.passes.graph.utils import deepcopy_mase_graph
from chop.tools.get_input import InputGenerator
from chop.tools.checkpoint_load import load_model
from chop.ir import MaseGraph

from chop.models import get_model_info, get_model

set_logging_verbosity("info")

/opt/conda/envs/mase/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.1.3


INFO     Set logging level to info
I0306 20:07:56.248935 140074075768640 logger.py:44] Set logging level to info


In [2]:
model_name = "jsc-tiny"
dataset_name = "jsc"
max_epochs = 1
batch_size = 256
learning_rate = 1e-3
accelerator = "gpu"

data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
)
data_module.prepare_data()
data_module.setup()

# 📝️ change this CHECKPOINT_PATH to the one you trained in Lab1
CHECKPOINT_PATH = "/root/mase/TensorRTDev/jsc-tiny_classification_jsc_2024-03-05/software/training_ckpts/best.ckpt"
#CHECKPOINT_PATH = "/root/mase/TensorRTDev/jsc-trt_classification_jsc_2024-03-05/software/training_ckpts/best.ckpt"
#CHECKPOINT_PATH = "/root/mase/TensorRTDev/jsc-toy_classification_jsc_2024-03-05/software/training_ckpts/best.ckpt"

model_info = get_model_info(model_name)
# quant_modules.initialize()
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False)

model = load_model(load_name=CHECKPOINT_PATH, load_type="pl", model=model)

INFO     Loaded pytorch lightning checkpoint from /root/mase/TensorRTDev/jsc-tiny_classification_jsc_2024-03-05/software/training_ckpts/best.ckpt
I0306 20:08:01.080414 140074075768640 checkpoint_load.py:85] Loaded pytorch lightning checkpoint from /root/mase/TensorRTDev/jsc-tiny_classification_jsc_2024-03-05/software/training_ckpts/best.ckpt


In [3]:
mg = MaseGraph(model=model)

In [4]:
import toml

# Path to your TOML file
toml_file_path = '/root/mase/machop/configs/examples/jsc_trt_quantization.toml'

# Reading TOML file and converting it into a Python dictionary
with open(toml_file_path, 'r') as toml_file:
    pass_args = toml.load(toml_file)

# Extract the 'passes.tensorrt-calibrate' section and its children
tensorrt_calibrate_config = pass_args.get('passes', {}).get('tensorrt-calibrate', {})
# Extract the 'passes.tensorrt-quantize' section and its children
tensorrt_quantize_config = pass_args.get('passes', {}).get('tensorrt-quantize', {})
# Extract the 'passes.tensorrt-train' section and its children
tensorrt_train_config = pass_args.get('passes', {}).get('tensorrt-train', {})
# Extract the 'passes.tensorrt-analysis' section and its children
tensorrt_analysis_config = pass_args.get('passes', {}).get('tensorrt-analysis', {})

# Print or return the extracted section
print("TensorRT Quantize Config:", tensorrt_calibrate_config)
print("TensorRT Train Config:", tensorrt_train_config)
print("TensorRT Quantize Config:", tensorrt_quantize_config)
print("TensorRT Analysis Config:", tensorrt_analysis_config)

TensorRT Quantize Config: {'by': 'type', 'calibrator': ['percentile', 'max', 'mse', 'entropy'], 'percentiles': [99], 'report': True, 'linear': {'config': {'quantize': True}, 'input': {'calibrator': 'histogram', 'quantize_axis': False}, 'weight': {'calibrator': 'histogram', 'quantize_axis': False}}, 'default': {'config': {'quantize': True, 'input': {'calibrator': 'histogram', 'quantize_axis': False}, 'weight': {'calibrator': 'histogram', 'quantize_axis': False}}}}
TensorRT Train Config: {}
TensorRT Quantize Config: {'by': 'type', 'default': {'config': {'name': 'integer', 'data_in_width': 8, 'data_in_frac_width': 4, 'weight_width': 8, 'weight_frac_width': 9, 'bias_width': 8, 'bias_frac_width': 9}}}
TensorRT Analysis Config: {'analysis': ['latency', 'accuracy', 'gpu_power', 'F1_score', 'recall', 'precision'], 'num_batches': 10}


In [5]:
task = 'classification'

# get the input generator
input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task=task,
    which_dataloader="train",
)

train_generator = InputGenerator(
    model_info=model_info,
    data_module=data_module,
    task=task,
    which_dataloader="train",
)

val_generator = InputGenerator(
    model_info=model_info,
    data_module=data_module,
    task=task,
    which_dataloader="val",
)

configs = [tensorrt_calibrate_config, tensorrt_train_config, tensorrt_quantize_config, tensorrt_analysis_config]
for config in configs:
    # config["train_generator"] = train_generator
    # config["val_generator"] = val_generator
    config['batch_size'] = pass_args['batch_size']
    config['data_module'] = data_module
    config['accelerator'] = 'cuda' if pass_args['accelerator'] == 'gpu' else pass_args['accelerator']
    if config['accelerator'] == 'gpu':
        os.environ['CUDA_MODULE_LOADING'] = 'LAZY'

# a demonstration of how to feed an input value to the model
dummy_in = next(iter(input_generator))
_ = model(**dummy_in)

mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)

In [6]:
# for inputs in data_module.train_dataloader():
#     xs, ys = inputs
#     print(ys.shape)
#     break

In [8]:
'''  
./ch transform --config configs/examples/jsc_trt_quantization.toml --load ../TensorRTDev/jsc-tiny_classification_jsc_2024-03-05/software/training_ckpts/best.ckpt --load-type pl
./ch transform --config configs/examples/jsc_trt_quantization.toml --load ../TensorRTDev/jsc-trt_classification_jsc_2024-03-05/software/training_ckpts/best.ckpt --load-type pl
./ch transform --config configs/examples/jsc_trt_quantization.toml --load ../TensorRTDev/jsc-toy_classification_jsc_2024-03-05/software/training_ckpts/best.ckpt --load-type pl
'''
import copy
# Restart kernel and re-run previous cells to avoid any issues
# mg_og = deepcopy_mase_graph(mg)
mg_og = copy.deepcopy(mg)

# Calibrate model (passing data samples to the quantizer and deciding the best amax for activations)
mg, _ = tensorrt_calibrate_transform_pass(mg, pass_args=tensorrt_calibrate_config)

# # Conduct QAT
# mg, _ = tensorrt_train_transform_pass(mg, pass_args=tensorrt_train_config)

# Convert and store to ONNX and then TensorRT
mg, trt_dict = tensorrt_quantize_transform_pass(mg, pass_args=tensorrt_quantize_config)

# Compare original tensorrt with quantized graph
mg, _ = tensorrt_analysis_pass(mg_og, trt_dict['trt_graph_path'], pass_args=tensorrt_analysis_config)

# Analysis pass
...

Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 577, in _pydevd_bundle.pydevd_cython.PyDBFrame._handle_exception
  File "_pydevd_bundle/pydevd_cython.pyx", line 312, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "/opt/conda/envs/mase/lib/python3.11/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2070, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/envs/mase/lib/python3.11/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2106, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


RuntimeError: Only Tensors created explicitly by the user (graph leaves) support the deepcopy protocol at the moment.  If you were attempting to deepcopy a module, this may be because of a torch.nn.utils.weight_norm usage, see https://github.com/pytorch/pytorch/pull/103001